In [1]:
# =========================
# Standard library
# =========================
import collections
import datetime
import io
import math
import os
import shutil
import textwrap
import warnings

# =========================
# Third-party libraries
# =========================
# Data science
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Images / PDF
import fitz  # PyMuPDF
from PIL import Image
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfMerger

# Report generation (ReportLab)
from reportlab.lib import colors
from reportlab.lib.pagesizes import A1, A3, A4, portrait, landscape
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import mm
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase.pdfmetrics import registerFont
from reportlab.pdfgen import canvas
from reportlab.platypus import Table, TableStyle, Paragraph, ListFlowable, ListItem
from reportlab.pdfbase.ttfonts import TTFont

# Excel
import openpyxl
from openpyxl.drawing.image import Image as open_Image
from openpyxl.styles import PatternFill

# Utilities
from tqdm.notebook import tqdm

# =========================
# Local packages
# =========================
from lib.data.db import *
from lib.data.course_data import *
from lib.sql import *
from lib.setting import *

# =========================
# Runtime config
# =========================
warnings.simplefilter('ignore')
sns.set(font='Hiragino Sans')
# HeiseiKakuGo-W5 をPDF用に登録
pdfmetrics.registerFont(UnicodeCIDFont("HeiseiKakuGo-W5"))

# （Jupyter 環境の場合のみ）
%matplotlib inline

# # オプション:
# # import japanize_matplotlib
# # sns.set_style('whitegrid')


# 概要

--作業内容--

1. コンピデータ確認
2. プロット作成
3. p3(コンピページ作成)
4. p2部分のExcelに代入
    1. コピー作成
    2. 一つ一つ開いて代入
    3. 保存してPDF保存
4. 元々のやつと合体

# データチェック

In [2]:
# データ読み込み
score = pd.read_csv("import/F1040-03_2024-11-07 00:00:00_2024年度360度評価_補正済みスコア.csv")
flag_1 = pd.read_excel("import/所属追加_F1040名古屋鉄道株式会社様_ユーザーリスト.xlsx",sheet_name="1.ユーザー情報",dtype={"社員番号":str})

# 必要フラグ = [['姓', '名','社員番号',"所属","役職分類\n(社内)"]]
flag_1 = flag_1[['姓', '名','社員番号',"所属","役職分類\n(社内)"]]
# 所属.notna()
flag_1 = flag_1[flag_1["所属"].notna()].reset_index(drop=True)
# 氏名作成
flag_1["氏名"] = flag_1["姓"] + flag_1["名"]


# merge()
meitestsu_dat = flag_1.merge(score,on="氏名")

# データ数チェック
print("flag_data数",flag_1.shape)
print("merge後",meitestsu_dat.shape)

# columns整理
meitestsu_dat.rename(columns={"flag_id":"Evaluatee Flag ID"},inplace=True)

flag_data数 (489, 6)
merge後 (489, 49)


In [3]:
# 名鉄測定項目
meitetsu_cn = ['課題設定', '創造性', '論理的思考', '個人的実行力', 'ヴィジョン',
       '成長', '耐性', '感情コントロール', '決断力', '共感・傾聴力', '外交性', '影響力の行使', '情熱・宣教力',
       '組織への働きかけ', '誠実さ']
meitetsu_cn_self = ['課題設定（自己評価）', '創造性（自己評価）', '論理的思考（自己評価）',
       '個人的実行力（自己評価）', 'ヴィジョン（自己評価）', '成長（自己評価）', '耐性（自己評価）', '感情コントロール（自己評価）',
       '決断力（自己評価）', '共感・傾聴力（自己評価）', '外交性（自己評価）', '影響力の行使（自己評価）',
       '情熱・宣教力（自己評価）', '組織への働きかけ（自己評価）', '誠実さ（自己評価）']

# 自己他者gap
sel_oth_gap_list = [i+"_sel_oth_gap" for i in meitetsu_cn]

for i,j in zip(meitetsu_cn,meitetsu_cn_self):
    meitestsu_dat[f"{i}_sel_oth_gap"] = meitestsu_dat[i] - meitestsu_dat[j]

In [4]:
# 等級ごと平均値算出
# グレード調整
"""
【G8】Ｍ１      27
ーーーーーーーーーーーーーーーーーー
【G7】Ｍ２      68
ーーーーーーーーーーーーーーーーーー
【G6】Ｍ３     125／ＧＭ３      1
ーーーーーーーーーーーーーーーーーー
【G4】Ｍ４     116
ーーーーーーーーーーーーーーーーーー
【G2】ＭＳ１    118／ＭＳ２     34
"""
tmp_dict = {
    "ＧＭ３":"Ｍ３/ＧＭ３",
    "Ｍ３":"Ｍ３/ＧＭ３",
    "ＭＳ２":"ＭＳ１/２",
    "ＭＳ１":"ＭＳ１/２",
}
meitestsu_dat["grade_IGS調整"] = meitestsu_dat["役職分類\n(社内)"].replace(tmp_dict)

grade_median_table = meitestsu_dat.groupby("grade_IGS調整")[meitetsu_cn].median()

In [5]:
grade_median_table

,課題設定,創造性,論理的思考,個人的実行力,ヴィジョン,成長,耐性,感情コントロール,決断力,共感・傾聴力,外交性,影響力の行使,情熱・宣教力,組織への働きかけ,誠実さ
grade_IGS調整,,,,,,,,,,,,,,,
Ｍ１,75.310345,65.666501,79.662628,77.444457,71.615830,66.978728,81.460344,77.199423,80.327408,68.176249,83.690600,79.267858,77.463700,77.204821,81.857710
Ｍ２,78.276415,64.648386,75.585983,75.715986,70.898229,66.887935,80.367661,74.033936,76.904053,70.616040,80.015755,74.669502,72.251054,73.680235,79.419497
Ｍ３/ＧＭ３,73.380062,61.159493,75.636751,75.355481,66.794820,62.752244,79.099407,72.300425,77.019292,69.014849,76.075121,69.333594,71.968014,72.298752,78.635838
Ｍ４,72.637377,63.469438,72.645791,77.707196,63.869361,67.645683,79.082025,75.613464,74.452290,70.181658,75.084131,66.197107,72.169681,73.432786,75.397135
ＭＳ１/２,61.823794,58.086901,62.057199,76.108663,56.817244,68.948031,75.892799,74.875042,65.840273,69.623108,73.205595,52.659036,68.922099,69.136808,66.289789


# P3作成

In [6]:
meitestsu_dat["役職分類\n(社内)"].value_counts().sort_index()

役職分類\n(社内)
ＧＭ３      1
Ｍ１      27
Ｍ２      68
Ｍ３     125
Ｍ４     116
ＭＳ１    118
ＭＳ２     34
Name: count, dtype: int64

In [7]:
score.columns

Index(['flag_id', 'flag_id.1', '氏名', '年代', '所属1', '所属2', '所属3', '所属4', '所属5',
       '役職種別id', '役職クラスid', '役職', '課題設定', '創造性', '論理的思考', '個人的実行力', 'ヴィジョン',
       '成長', '耐性', '感情コントロール', '決断力', '共感・傾聴力', '外交性', '影響力の行使', '情熱・宣教力',
       '組織への働きかけ', '誠実さ', '課題設定（自己評価）', '創造性（自己評価）', '論理的思考（自己評価）',
       '個人的実行力（自己評価）', 'ヴィジョン（自己評価）', '成長（自己評価）', '耐性（自己評価）', '感情コントロール（自己評価）',
       '決断力（自己評価）', '共感・傾聴力（自己評価）', '外交性（自己評価）', '影響力の行使（自己評価）',
       '情熱・宣教力（自己評価）', '組織への働きかけ（自己評価）', '誠実さ（自己評価）', 'deadline_date',
       'course_id'],
      dtype='object')

## プロット作成

### ベンチマークデータ調整

In [8]:
# bench_読み込み
dat_Bench_max100 = pd.read_csv("import/99_dat_Bench_max100.csv")

In [9]:
# # V2でこのセル丸々置き換え必須
# tmp_table = pd.DataFrame(index = cn,columns=["-1標準偏差","中央閾値","+1標準偏差",])
# tmp_table["-1標準偏差"] = 45
# tmp_table["中央閾値"] = 55
# tmp_table["+1標準偏差"] = 65
# tmp_table = tmp_table.T

# ベンチ5階層ごとにtable異なるので要注意

# bench_1 = G8
# bench_2 = G7
# bench_3 = G6
# bench_4 = G4
# bench_5 = G2

# bench_1
bench_1_tmp = dat_Bench_max100[dat_Bench_max100["役職分類\n(社内)"] == "部長クラス"][cn].describe()
bench_1_tmp_2 = bench_1_tmp.loc[["50%"]].rename(index={"50%": "中央閾値"})

bench_1_tmp_2.loc["+1標準偏差"] = bench_1_tmp_2.loc["中央閾値"] + bench_1_tmp.loc["std"]
bench_1_tmp_2.loc["-1標準偏差"] = bench_1_tmp_2.loc["中央閾値"] - bench_1_tmp.loc["std"]

bench_1 = bench_1_tmp_2

# bench_2
bench_2_tmp = dat_Bench_max100[dat_Bench_max100["役職分類\n(社内)"] == "部長クラス"][cn].describe()
bench_2_tmp_2 = bench_2_tmp.loc[["50%"]].rename(index={"50%": "中央閾値"})

bench_2_tmp_2.loc["+1標準偏差"] = bench_2_tmp_2.loc["中央閾値"] + bench_2_tmp.loc["std"]
bench_2_tmp_2.loc["-1標準偏差"] = bench_2_tmp_2.loc["中央閾値"] - bench_2_tmp.loc["std"]

bench_2 = bench_2_tmp_2

# bench_3
bench_3_tmp = dat_Bench_max100[dat_Bench_max100["役職分類\n(社内)"] == "課長クラス"][cn].describe()
bench_3_tmp_2 = bench_3_tmp.loc[["50%"]].rename(index={"50%": "中央閾値"})

bench_3_tmp_2.loc["+1標準偏差"] = bench_3_tmp_2.loc["中央閾値"] + bench_3_tmp.loc["std"]
bench_3_tmp_2.loc["-1標準偏差"] = bench_3_tmp_2.loc["中央閾値"] - bench_3_tmp.loc["std"]

bench_3 = bench_3_tmp_2

# bench_4
bench_4_tmp = dat_Bench_max100[dat_Bench_max100["役職分類\n(社内)"] == "主任・係長クラス"][cn].describe()
bench_4_tmp_2 = bench_4_tmp.loc[["50%"]].rename(index={"50%": "中央閾値"})

bench_4_tmp_2.loc["+1標準偏差"] = bench_4_tmp_2.loc["中央閾値"] + bench_4_tmp.loc["std"]
bench_4_tmp_2.loc["-1標準偏差"] = bench_4_tmp_2.loc["中央閾値"] - bench_4_tmp.loc["std"]

bench_4 = bench_4_tmp_2

# bench_5
bench_5_tmp = dat_Bench_max100[dat_Bench_max100["役職分類\n(社内)"] == "一般クラス"][cn].describe()
bench_5_tmp_2 = bench_5_tmp.loc[["50%"]].rename(index={"50%": "中央閾値"})

bench_5_tmp_2.loc["+1標準偏差"] = bench_5_tmp_2.loc["中央閾値"] + bench_5_tmp.loc["std"]
bench_5_tmp_2.loc["-1標準偏差"] = bench_5_tmp_2.loc["中央閾値"] - bench_5_tmp.loc["std"]

bench_5 = bench_5_tmp_2



# グレードで読み出せるように設定
compare_bench_dict = {
    "Ｍ１":bench_1,
    "Ｍ２":bench_2,
    "Ｍ３":bench_3,
    "Ｍ３/ＧＭ３":bench_3,
    "Ｍ４":bench_4,
    "ＭＳ１":bench_5,
    "ＭＳ２":bench_5,
    "ＭＳ１/２":bench_5,
    "ＧＭ３":bench_3,
}

### ベンチマーク比較

In [10]:
# レーダーチャート用

radar_cn = [
 '課題設定',
 # '解決意向',
 '創造性',
 '論理的思考',
 # '疑う力',
    
 '個人的実行力',
 # '内的価値',
 'ヴィジョン',
 # '自己効力',
 '成長',
 # '興味',
 '耐性',
 '感情\nコントロール', # ここだけ折り返し
 '決断力',
    
 # '表現力',
 '共感・傾聴力',
 '外交性',
 # '柔軟性',
 # '寛容',
 '影響力の行使',
 '情熱・宣教力',
    
 '組織への働きかけ',
 # '地球市民',
 # '組織へのコミットメント',
 '誠実さ'
]

In [11]:
cog_val = [100,100,100,100,100,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100]
sel_val = [0,0,0,0,0,100,100,100,100,100,100,100,100,100,100,0,0,0,0,0,0,0,0,0,0,0]
oth_val = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,100,100,100,100,100,100,100,0,0,0,0]
com_val = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,100,100,100,100]


cn
# self_cols=自己評価カラム名
# cn=他者評価カラム名

y=dict(boxstyle="round",
                   ec="#F4CF39",
                   fc="#F4CF39",
                   )
r=dict(boxstyle="round",
                   ec="#DD5556",
                   fc="#DD5556",
                   )
g=dict(boxstyle="round",
                   ec="#13A57E",
                   fc="#13A57E",
                   )
b=dict(boxstyle="round",
                   ec="#427FC2",
                   fc="#427FC2",
                   )
bboxc = [y,y,y,r,r,r,r,r,r,g,g,g,g,b,b]

#ベンチマークレーダー作成
def bench_radar_meitestsu(data,col=cn,radar_col = radar_cn):
    """
    data = dataset with everything
    ID = examinee ID
    fname = filepath
    """
    # フォルダ作成
    os.makedirs("materials/基準値比較レーダーチャート", exist_ok=True)

    adj = 0.125663705

    all_ids = data["Evaluatee Flag ID"].tolist()

    for id_ in tqdm(all_ids, total=len(all_ids)):
        row = data[data["Evaluatee Flag ID"]==id_].squeeze()
        name = row["Evaluatee Flag ID"]
        
        # gradeによってデータ選択
        grade = row["役職分類\n(社内)"]
        # ベンチマークデータ調整で作ったdictから読み込み
        bench_dat = compare_bench_dict[row["役職分類\n(社内)"]]


        val1, val2, val3, val4  = np.array(row[col]), np.array(bench_dat.loc["-1標準偏差"][col]),np.array(bench_dat.loc["中央閾値"][col]),np.array(bench_dat.loc["+1標準偏差"][col])
        theta = np.linspace(0, 2 * np.pi, 16, endpoint=True)
        adj_theta = np.linspace(0-adj, 2 * np.pi, 16, endpoint=True)

        val1, val2, val3, val4 = np.concatenate((val1,[val1[0]])), np.concatenate((val2,[val2[0]])), np.concatenate((val3,[val3[0]])), np.concatenate((val4,[val4[0]]))
        fig = plt.figure(figsize=(13,12))
        plt.subplots_adjust(top=0.8,hspace=0.8)
        ax = fig.add_subplot(111,polar=True)

    #GROW360のレポートに合わせて反時計回りに並べる
        ax.plot(-theta,val1,'o-', label="あなたの結果",color="#13A57E",alpha=0.5)
        ax.plot(-theta,val4,'o-', label=f"市場水準・上位",color="#000000",alpha=0.7,lw=5)
        ax.plot(-theta,val3,'--', label=f"市場水準・中位",color="#000000",alpha=0.5,lw=3)
        ax.plot(-theta,val2,'--', label=f"市場水準・下位",color="#000000",alpha=0.5,lw=1.5)


        ax.fill(-theta, val1, alpha=0.5,color="#13A57E")

        ax.set_rlim(0 ,110)
        ax.set_rgrids(np.arange(0,120,20),angle=0,fontsize=10)
        ax.set_thetagrids(np.degrees(theta)[:-1],[],fontsize=10)  # 軸ラベル

        for i in range(len(radar_col)):
            ax.text(-theta[i], 116, radar_col[i], ha="center", va="center", fontsize=12.5, bbox=bboxc[i],
                    fontweight="bold",color="white")

        ax.set_theta_direction(-1)
        ax.set_theta_zero_location("N")
        matplotlib.rc('axes',edgecolor='lightgray', linewidth=2)
        plt.legend(loc="upper left", bbox_to_anchor=(0.9, 1.05), borderaxespad=0, fontsize=12.5)

        plt.savefig(f"materials/基準値比較レーダーチャート/{name}_基準値比較",dpi=200,pad_inches=0.2,bbox_inches="tight")
        plt.close()

In [12]:
bench_radar_meitestsu(meitestsu_dat.head(),col = meitetsu_cn)

  0%|          | 0/5 [00:00<?, ?it/s]

### 自己他者乖離

In [13]:
# 自己他者乖離
def self_other_radar_meitestsu(data,col=cn,radar_col = radar_cn,col_self = cn_self):
    """
    data = dataset with everything
    ID = examinee ID
    fname = filepath
    """
    # フォルダ作成
    os.makedirs("materials/自己他者比較レーダーチャート", exist_ok=True)

    adj = 0.125663705
    all_ids = data["Evaluatee Flag ID"].tolist()
    for id_ in tqdm(all_ids, total=len(all_ids)):
        row = data[data["Evaluatee Flag ID"]==id_].squeeze()
        name = row["Evaluatee Flag ID"]

        val1, val2  = np.array(row[col_self]), np.array(row[col])#, np.array(bigdata[col_engi].mean().round(0))

        theta = np.linspace(0, 2 * np.pi, 16, endpoint=True)
        adj_theta = np.linspace(0-adj, 2 * np.pi, 16, endpoint=True)

        val1, val2 = np.concatenate((val1,[val1[0]])), np.concatenate((val2,[val2[0]]))#, np.concatenate((val3,[val3[0]]))
        fig = plt.figure(figsize=(12,12))
        plt.subplots_adjust(top=0.8,hspace=0.8)
        ax = fig.add_subplot(111,polar=True)

    #GROW360のレポートに合わせて反時計回りに並べる
        ax.plot(-theta,val1,'o-', label="あなたの自己測定",color="#DD5556")
        ax.plot(-theta,val2,'s-', label="あなたの結果",color="#13A57E")

        ax.fill(-theta, val1, alpha=0.5,color="#DD5556")
        ax.fill(-theta, val2, alpha=0.5,color="#13A57E")


        ax.set_rlim(0 ,110)
        ax.set_rgrids(np.arange(0,120,20),angle=0,fontsize=10)
        ax.set_thetagrids(np.degrees(theta)[:-1],[],fontsize=10)  # 軸ラベル

        for i in range(len(radar_col)):
            ax.text(-theta[i], 116, radar_col[i], ha="center", va="center", fontsize=12.5, bbox=bboxc[i],
                    fontweight="bold",color="white")

        ax.set_theta_direction(-1)
        ax.set_theta_zero_location("N")
        matplotlib.rc('axes',edgecolor='lightgray', linewidth=2)
        plt.legend(loc="upper left", bbox_to_anchor=(0.9, 1.05), borderaxespad=0, fontsize=12.5)

        plt.savefig(f"materials/自己他者比較レーダーチャート/{name}_自己他者比較",dpi=200,pad_inches=0.2,bbox_inches="tight")
        plt.close()

In [14]:
self_other_radar_meitestsu(meitestsu_dat.head(),col = meitetsu_cn,col_self = meitetsu_cn_self)

  0%|          | 0/5 [00:00<?, ?it/s]

## 説明文辞書作成

In [15]:
comp_desc = {
    "課題設定":"課題を分析し原因を突き止めている",
    "解決意向":"課題に合わせた計画や方法を立案している",
    "創造性":"独創的なアイデアを生み出している",
    "論理的思考":"物事について論理的に考えている",
    "疑う力":"情報を鵜呑みにせずに自ら考えている",
    "個人的実行力":"何事も最後までやり遂げている",
    "内的価値":"一貫した価値観をもって判断している",
    "ヴィジョン":"将来の変化を予測して目標や計画を立てている",
    "自己効力":"自信をもって物事に取り組んでいる",
    "成長":"自身の成長を意識して物事に取り組んでいる",
    "興味":"様々な分野で知見を深めようとしている",
    "耐性":"困難な状況でもポジティブに物事に取り組んでいる",
    "感情コントロール":"ストレスを感じる状況でも行動をコントロールしている",
    "決断力":"適切なタイミングで、責任をもって物事を決めている",
    "表現力":"物事をわかりやすく伝えている",
    "共感・傾聴力":"相手の意図や感情を的確に理解している",
    "外交性":"様々な場面で人脈を構築している",
    "柔軟性":"臨機応変に物事に対応している",
    "寛容":"自分と異なる意見や行動を受け入れている",
    "影響力の行使":"リーダーシップを発揮している",
    "情熱・宣教力":"熱意を示すことで周囲の協力を得ている",
    "組織への働きかけ":"チームワークを育む行動をしている",
    "地球市民":"グローバルな問題に関心を示している",
    "組織へのコミットメント":"所属する組織の目的達成に向け真摯に取り組んでいる",
    "誠実さ":"正しく倫理的な行いをするように周囲に働きかけている",
    "-": "-"
}

In [16]:
comp_dev_dict = {
    "課題設定":"問題の本質を見極めながら課題を設定する / 問題を多角的に分析して原因を突き止める / 情報収集の方法を理解し、その情報を課題の設定に生かす / 日頃から周囲の問題に関心をもち、解決すべき課題はないか考える",
    "解決意向":"メリットとデメリットを踏まえながら実現することが可能な計画を立てるようにする / 計画に問題があれば改善につなげる修正を積極的に行う / 課題ときちんと適合した具体的な目標を設定する習慣を付ける",
    "創造性":"周囲に自分独自の価値を実際に示す / 周囲が思い付かないような多数のアイデアを提案する / 既存の複数のアイデアを一つに統合して新たなアイデアや価値として提案し、実現への道筋を示すようにする",
    "論理的思考":"物事の因果関係を矛盾なく整理する / 論理の飛躍や矛盾に敏感になり、それについて質問・指摘するようにする / 抽象度の高い複雑な物事でも、個々の要素に分解して具体的にイメージして考えるようにする",
    "疑う力":"他者の言動に疑問をもち、言動の背景を突き止めるようにする / 情報の発信源や入手経路を調べ、事実と意見を区別して情報の正しさを判断する / 相手が誰であれ、その人の主張に誤りがあるのではないかと考える",
    "個人的実行力":"何があっても諦めずに最後までやり遂げるようにする / 困難なことであっても、リスクを考慮しながら失敗を恐れずに挑戦してみる / あらゆる手段を尽くし、自己犠牲もいとわないようにする",
    "内的価値":"明確な信念や価値観をもち、それに従って一貫した行動をとる / 判断のよりどころとなる明確な信念や価値観をもつ / 自らの強固な信念や価値観に沿って、終始一貫した意思決定を行うようにする",
    "ヴィジョン":"将来起こりうる社会変化を分析し、実現性の高い具体的なイメージをもつようにする / 将来の夢や理想を明確にもち、自分がワクワクするだけでなく、自信をもって周囲を巻き込めるように働きかける",
    "自己効力":"自分はほかの人と比べて高いレベルで物事を遂行できると自信をもつようにする / 困難な出来事が起こっても、過去にやり抜いてきた自分をよりどころにしながら自信をもってそれに立ち向かい、やり切るようにする",
    "成長":"自分を成長させるために高い目標を立て、必要なら厳しい道であってもあえて選択してみる / 高い向上心をもち、自分のさらなる成長につながる機会を積極的に求める / 現状に満足せず、高い向上心をもつようにする",
    "興味":"新しいものに触れることに喜びを感じ、積極的に行動する / 情報や知識を仕入れる手段を多くもち、それを活用しながら情報や知識を集めるようにする / 知らないことを追い求めることに喜びを感じ、関心をもったことを突き詰めて考える",
    "耐性":"どんなに困難な状況でもポジティブに考えるようにする / 苦しい状況でも周囲に辛い顔を見せず、状況が好転するまで粘り強く耐えるようにする / 失敗や挫折に対して、その状況を成長の糧にして、前向きに捉える",
    "感情コントロール":"ストレスの原因を分析し、複数の対処法を使い分ける術をもつ / 必要なときには回避する勇気をもつ / 自分の感情の状態を自覚し、平常時との違いを認識しながら思考や行動を修正するようにする",
    "決断力":"状況が複雑で判断に迷っても先延ばしせず、迅速に決断する / 自分の決断に対する責任を全うし、対応が困難であっても挽回するために尽力する / 自分が正しいと信じている思考や感情に基づいて決断する",
    "表現力":"五感を用いて表現し、相手にその感覚を伝えられるようにする / ジェスチャーや図表などを効果的に用いて、伝えるべきことをより効果的に伝えるようにする / 相手の知性と感情の両方に訴えかけるようにしてみる",
    "共感・傾聴力":"話の内容以外に相手の表情や仕草にも注意を払い、相手の考えを推測する / 相手の話をきちんと受け止め、相手の考えや気持ちに共感を示し、気分良く話をさせるように心掛ける / 相手やその話に対して自分のことであるかのように共感する",
    "外交性":"どんな場面・相手でも会話を楽しみ、周囲の人たちと打ち解けられるようにする / 新しい人間関係を構築できるように自ら積極的に働きかける / 初対面の人であっても自分から積極的に話しかけ、その人の考えを知ろうする",
    "柔軟性":"環境に変化が起こった場合は積極的にやり方を改めて対応する / これまでの実績や固定観念を否定する勇気をもち、異なるアプローチで挑む / 変化に富む環境にあえて身を置き、その環境を楽しむようにする",
    "寛容":"異なる意見がなされた理由を理解したうえで、それを受け入れるようにする / 他者の成功を自分ごとのように喜び、その気持ちを相手にも伝える / 間違いや失敗を犯しても、それを受け止めたうえで相手を心から許す",
    "影響力の行使":"どんなグループでもリーダーシップを取り、グループを率先してまとめる / 相手の意見や行動に影響を与えられるように働きかける / どうすれば自分の言動が相手の態度・行動を変えることができるのかを考えてみる",
    "情熱・宣教力":"自分の力だけではどうすることもできないことに対しては周囲に協力を求め、決して諦めることなく他者を巻き込みながらそれをやり遂げるようにする",
    "組織への働きかけ":"グループの目標や自分の役割を理解し、メンバーが意欲的に動けるような働きかけをする / 建設的な発言を積極的に行い、議論の活性化に貢献する/求められる役割だけでなく、自分がどう動けば効果的かを考えて行動する",
    "地球市民":"異文化の人との協働に積極的に参加する /人種、国籍、思想、文化、宗教などの枠組みにとらわれず、自分の役割を地球レベルで捉えて行動する / 地域や日本のために自分にできることはないか模索し、実際に行動してみる",
    "組織へのコミットメント":"自分が所属するチームやネットワークに愛着をもち、その組織を維持・拡大させるために、自分の利益より優先して力を尽くす",
    "誠実さ":"約束を守る意味を理解したうえで、自ら積極的に約束を守る / 自分に都合が悪い状況でも、事の善悪を正しく判断して行動する / ルールやマナーを守らない人に働きかけ、相手の態度や行動を少しでも変えられるように努力する",
    "-":"-"
}

## page3

### コンピ部分

In [17]:
TTFont

reportlab.pdfbase.ttfonts.TTFont

In [18]:
def set_info(filename):
    a3_width, a3_height = A3
    custom_size = (a3_width - 50, a3_height)  # 縦を50ポイント縮める
    pdf_canvas = canvas.Canvas("{0}.pdf".format(filename), pagesize=landscape(custom_size))
    # pdf_canvas = canvas.Canvas("{0}.pdf".format(filename),pagesize=landscape(A3)) #A3横設定
    #memo canvas.Canvas(file_path)で新規PDFを作成

    # ファイル情報の登録（任意）
    pdf_canvas.setAuthor("") # 作者
    pdf_canvas.setTitle("") # 表題
    pdf_canvas.setSubject("") # 件名
    return pdf_canvas


def make_page3(df , language="Japanese",cn=cn):# ファイル名の設定
    now = datetime.datetime.now()
    # フォルダ作成
    os.makedirs("workspace/page3/", exist_ok=True)

    for index,data in tqdm(df.iterrows(), total=len(df)):
        ID = data["Evaluatee Flag ID"]
        filename="workspace/page3/{0}_page3".format(ID,now)#memo IDに入れたものが{0}に入る
        pdf_canvas = set_info(filename) # キャンバス名の設定
        print_string_page3(data, ID, language, pdf_canvas,cn=cn)
        pdf_canvas.save() # pdfを保存

In [19]:
def print_string_page3(df, ID, language, pdf_canvas,cn=cn):
    # (1)タイトル
    font_size = 10# フォントサイズ
    font = 'HeiseiKakuGo-W5'
    # pdf_canvas.setFont('HeiseiKakuGo-W5', font_size)
    pdf_canvas.setFont(font, font_size)
    
    
    row = df
    name = row["氏名"]
    # ベンチマークデータ調整で作ったdictから読み込み
    bench_dat = compare_bench_dict[row["役職分類\n(社内)"]]

    
    pdfmetrics.registerFont(UnicodeCIDFont('HeiseiKakuGo-W5')) # フォント
    

        #セクション1タイトル
    data = [
        ["", "市場水準から見た、あなたの強み・弱み", "市場水準とのギャップから見える強み・弱みは何かを理解しましょう。"]
    ]

    table = Table(data, colWidths=(6*mm, 100*mm, 42*mm), rowHeights= (6*mm))# size of box
    table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ('BOX', (0, 0), (0, -1), 1, colors.Color(red=(102.0/255),green=(102.0/255),blue=(102.0/255))),
        ('BACKGROUND', (0, 0), (0, -1), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('FONT', (0, 0), (0, -1), font, 15),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.white),
        ('FONT', (1, 0), (1, -1), font, 15),
        ('TEXTCOLOR', (1, 0), (1, -1), colors.black),
        ('FONT', (2, 0), (2, -1), font, 8),#italicにしたい
        ('TEXTCOLOR', (2, 0), (2, -1), colors.black),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ("BACKGROUND",(1,0),(2,-1),colors.white),
    ]))
    table.wrapOn(pdf_canvas, 10*mm, 264*mm)
    table.drawOn(pdf_canvas, 10*mm, 264*mm)

    external_cols = cn

    bench_gap_table = pd.DataFrame()
    bench_gap_list = []
    for i in cn:
        abc = row[i] - bench_dat.loc["中央閾値"][i]
        bench_gap_list.append(abc)
    bench_gap_table = pd.DataFrame(bench_gap_list,index=cn,columns=["bench_gap"])
    bench_gap_table = bench_gap_table.sort_values("bench_gap")
    bench_gap_table = bench_gap_table.T
    highest = bench_gap_table.T.sort_values("bench_gap", ascending=False)[:3].index.tolist()
    lowest = bench_gap_table.T.sort_values("bench_gap")[:3].index.tolist()


    # get high and low, and descriptions
    if language == "Japanese":
        widthnumber=33
        
        # 2回目変更必要
        data = [
            ["強みとなるコンピテンシー","どのような能力か"],
            [highest[0], textwrap.fill(comp_desc[highest[0]], width=widthnumber)],
            [highest[1], textwrap.fill(comp_desc[highest[1]], width=widthnumber)],
            [highest[2], textwrap.fill(comp_desc[highest[2]], width=widthnumber)],
            ["弱みとなるコンピテンシー",""],
            [lowest[0], ""],
            [lowest[1], ""],
            [lowest[2], ""],
        ]
        # data = [
        #     ["強みとなるコンピテンシー","どのような能力か"],
        #     ["-","-"],
        #     ["-","-"],
        #     ["-","-"],
        #     ["弱みとなるコンピテンシー",""],
        #     ["-","-"],
        #     ["-","-"],
        #     ["-","-"],
        # ]

    table = Table(data, colWidths=(65*mm, 128*mm), rowHeights= (10*mm,8*mm,8*mm,8*mm,10*mm,25*mm,25*mm,25*mm))# size of box
    table.setStyle(TableStyle([
        ('BOX', (0, 0), (-1, -1), 1, colors.black),
        ('FONT', (0, 0), (-1, -1), font, 12),
        ('FONT', (1, 5), (1, 7), font, 10),
        ('FONT', (1, 1), (1, 3), font, 10),

        ('ALIGN', (0, 0), (-1, -1), "LEFT"),

        ('BACKGROUND', (0, 0), (-1, 0), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),

        ('BACKGROUND', (0, 4), (-1, 4), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('TEXTCOLOR', (0, 4), (-1, 4), colors.white),

        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black)
    ]))

    table.wrapOn(pdf_canvas, 10*mm, 140*mm) # adjust position
    table.drawOn(pdf_canvas, 10*mm, 140*mm)

    if language == "Japanese":
        widthnumber=39
        
        # 2回目変更必要
        data = [
            ["どのような能力か"],
            ["開発のヒント"],
            [textwrap.fill(comp_desc[lowest[0]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[lowest[0]], width=widthnumber)],
            [textwrap.fill(comp_desc[lowest[1]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[lowest[1]], width=widthnumber)],
            [textwrap.fill(comp_desc[lowest[2]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[lowest[2]], width=widthnumber)],
        ]
        # data = [
        #     ["どのような能力か"],
        #     ["開発のヒント"],
        #     ["-"],
        #     ["-"],
        #     ["-"],
        #     ["-"],
        #     ["-"],
        #     ["-"],
        # ]

    table = Table(data, colWidths=(128*mm), rowHeights= (5*mm,5*mm,7*mm,18*mm,7*mm,18*mm,7*mm,18*mm,))# size of box
    table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, -1), font, 9),
        ('ALIGN', (0, 0), (-1, -1), "LEFT"),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('TEXTCOLOR', (0, 0), (0, 1), colors.white),
    ]))


    table.wrapOn(pdf_canvas, 75*mm, 140*mm) # adjust position
    table.drawOn(pdf_canvas, 75*mm, 140*mm)

    #セクション2タイトル
    data = [
        ["", "あなたが気づいていない強みと周囲に認識されていないポテンシャル", ""]
    ]

    table = Table(data, colWidths=(6*mm, 90*mm, 42*mm), rowHeights= (6*mm))# size of box
    table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ('BOX', (0, 0), (0, -1), 1, colors.Color(red=(102.0/255),green=(102.0/255),blue=(102.0/255))),
        ('BACKGROUND', (0, 0), (0, -1), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('FONT', (0, 0), (0, -1), font, 15),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.white),
        ('FONT', (1, 0), (1, -1), font, 15),
        ('TEXTCOLOR', (1, 0), (1, -1), colors.black),
        ('FONT', (2, 0), (2, -1), font, 8),#italicにしたい
        ('TEXTCOLOR', (2, 0), (2, -1), colors.black),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ("BACKGROUND",(1,0),(2,-1),colors.white),
    ]))
    table.wrapOn(pdf_canvas, 10*mm, 125*mm)
    table.drawOn(pdf_canvas, 10*mm, 125*mm)

    data = [
        ["自己測定とのギャップから見える強み・ポテンシャルは何かを理解しましょう。"]
    ]

    table = Table(data, colWidths=(42*mm), rowHeights= (6*mm))# size of box
    table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ('FONT', (0, 0), (-1, -1), font, 8),#italicにしたい
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
    ]))
    table.wrapOn(pdf_canvas, 16*mm, 120*mm)
    table.drawOn(pdf_canvas, 16*mm, 120*mm)

    external_cols = cn

    sel_oth_gap_table = pd.DataFrame()
    sel_oth_list = []
    for i in sel_oth_gap_list:
        abc = row[i]
        sel_oth_list.append(abc)

    sel_oth_gap_table = pd.DataFrame(sel_oth_list,index=cn,columns=["gap"])
    sel_oth_gap_table = sel_oth_gap_table.sort_values("gap")
    sel_oth_gap_table = sel_oth_gap_table.T

    highest = sel_oth_gap_table.T.sort_values("gap", ascending=False)[:3].index.tolist()
    lowest = sel_oth_gap_table.T.sort_values("gap")[:3].index.tolist()

    # print(sel_oth_gap_table)

    true_lowest = []
    for i in lowest:
        if sel_oth_gap_table[i][0] < 0:
            true_lowest.append(i)
        else :
            true_lowest.append("-")

    true_highest = []
    for i in highest:
        if sel_oth_gap_table[i][0] > 0:
            true_highest.append(i)
        else :
            true_highest.append("-")

    # print(true_highest)
    # get high and low, and descriptions
    if language == "Japanese":
        widthnumber=33

        data = [
            ["あなたが気づいていない強み","どのような能力か"],
            [true_highest[0], textwrap.fill(comp_desc[true_highest[0]], width=widthnumber)],
            [true_highest[1], textwrap.fill(comp_desc[true_highest[1]], width=widthnumber)],
            [true_highest[2], textwrap.fill(comp_desc[true_highest[2]], width=widthnumber)],
            ["周囲に認識されていないポテンシャル",""],
            [true_lowest[0], ""],
            [true_lowest[1], ""],
            [true_lowest[2], ""],
        ]

    table = Table(data, colWidths=(65*mm, 128*mm), rowHeights= (10*mm,8*mm,8*mm,8*mm,10*mm,25*mm,25*mm,25*mm))# size of box
    table.setStyle(TableStyle([
        ('BOX', (0, 0), (-1, -1), 1, colors.black),
        ('FONT', (0, 0), (-1, -1), font, 12),
        ('FONT', (0, 4), (0, 4), font, 10),
        ('FONT', (1, 5), (1, 7), font, 10),
        ('FONT', (1, 1), (1, 3), font, 10),

        ('ALIGN', (0, 0), (-1, -1), "LEFT"),

        ('BACKGROUND', (0, 0), (-1, 0), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),

        ('BACKGROUND', (0, 4), (-1, 4), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('TEXTCOLOR', (0, 4), (-1, 4), colors.white),

        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),

        ('GRID', (0, 0), (-1, -1), 0.25, colors.black)

    ]))


    table.wrapOn(pdf_canvas, 10*mm, 1*mm) # adjust position
    table.drawOn(pdf_canvas, 10*mm, 1*mm)

    if language == "Japanese":
        widthnumber=39

        data = [
            ["どのような能力か"],
            ["開発のヒント"],
            [textwrap.fill(comp_desc[true_lowest[0]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[true_lowest[0]], width=widthnumber)],
            [textwrap.fill(comp_desc[true_lowest[1]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[true_lowest[1]], width=widthnumber)],
            [textwrap.fill(comp_desc[true_lowest[2]], width=widthnumber)],
            [textwrap.fill(comp_dev_dict[true_lowest[2]], width=widthnumber)],
        ]

    table = Table(data, colWidths=(128*mm), rowHeights= (5*mm,5*mm,7*mm,18*mm,7*mm,18*mm,7*mm,18*mm,))# size of box
    table.setStyle(TableStyle([
        ('FONT', (0, 0), (-1, -1), font, 9),
        ('ALIGN', (0, 0), (-1, -1), "LEFT"),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ('GRID', (0, 0), (-1, -1), 0.25, colors.black),
        ('TEXTCOLOR', (0, 0), (0, 1), colors.white),
    ]))


    table.wrapOn(pdf_canvas, 75*mm, 1*mm) # adjust position
    table.drawOn(pdf_canvas, 75*mm, 1*mm)


    #セクション4タイトル
    data = [
        ["", "市場水準比較レーダーチャート", "コンピテンシー測定結果と市場水準を比較したレーダーチャートも踏まえ、\n日頃の行動を見直してみましょう。"]
    ]

    table = Table(data, colWidths=(6*mm, 80*mm, 42*mm), rowHeights= (6*mm))# size of box
    table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ('BOX', (0, 0), (0, -1), 1, colors.Color(red=(102.0/255),green=(102.0/255),blue=(102.0/255))),
        ('BACKGROUND', (0, 0), (0, -1), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('FONT', (0, 0), (0, -1), font, 15),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.white),
        ('FONT', (1, 0), (1, -1), font, 15),
        ('TEXTCOLOR', (1, 0), (1, -1), colors.black),

        ('FONT', (2, 0), (2, -1), font, 8),#italicにしたい
        ('TEXTCOLOR', (2, 0), (2, -1), colors.black),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ("BACKGROUND",(1,0),(2,-1),colors.white),
    ]))
    table.wrapOn(pdf_canvas, 210*mm, 264*mm)
    table.drawOn(pdf_canvas, 210*mm, 264*mm)

    # if (row["グレード"]=="ZLD1") | (row["グレード"]=="ZEX1") | (row["グレード"]=="ZLD2") | (row["グレード"]=="ZEX2") | (row["グレード"]=="ZLD3") | (row["グレード"]=="ZEX3"):
    #     pdf_canvas.drawInlineImage(f"materials/基準値比較レーダーチャート/{ID}_基準値比較.png",250*mm, 140*mm,width=150*mm, height=120*mm)
    # else:
    #     pdf_canvas.drawInlineImage(f"materials/基準値比較レーダーチャート/{ID}_基準値比較.png",250*mm, 140*mm,width=140*mm, height=120*mm)
    
    pdf_canvas.drawInlineImage(f"materials/基準値比較レーダーチャート/{ID}_基準値比較.png",250*mm, 140*mm,width=135*mm, height=120*mm)


    #セクション5タイトル
    data = [
        ["", "自己測定比較レーダーチャート", "コンピテンシー測定結果と自己測定を比較したレーダーチャートも踏まえ、\n日頃の行動を見直してみましょう。"]
    ]

    table = Table(data, colWidths=(6*mm, 80*mm, 42*mm), rowHeights= (6*mm))# size of box
    table.setStyle(TableStyle([
        ("ALIGN", (0,0), (-1,-1), "LEFT"),
        ('BOX', (0, 0), (0, -1), 1, colors.Color(red=(102.0/255),green=(102.0/255),blue=(102.0/255))),
        ('BACKGROUND', (0, 0), (0, -1), colors.Color(red=(55.0/255),green=(75.0/255),blue=(143.0/255))),
        ('FONT', (0, 0), (0, -1), font, 15),
        ('TEXTCOLOR', (0, 0), (0, -1), colors.white),
        ('FONT', (1, 0), (1, -1), font, 15),
        ('TEXTCOLOR', (1, 0), (1, -1), colors.black),

        ('FONT', (2, 0), (2, -1), font, 8),#italicにしたい
        ('TEXTCOLOR', (2, 0), (2, -1), colors.black),
        ('VALIGN', (0, 0), (-1, -1), "MIDDLE"),
        ("BACKGROUND",(1,0),(2,-1),colors.white),
    ]))
    table.wrapOn(pdf_canvas, 210*mm, 125*mm)
    table.drawOn(pdf_canvas, 210*mm, 125*mm)

    pdf_canvas.drawInlineImage(f"materials/自己他者比較レーダーチャート/{ID}_自己他者比較.png",250*mm, 1*mm,width=135*mm, height=120*mm)



    pdf_canvas.showPage()
    

In [20]:
# from reportlab.pdfbase import pdfmetrics
# from reportlab.pdfbase.cidfonts import UnicodeCIDFont
# # フォント登録
# pdfmetrics.registerFont(UnicodeCIDFont("HeiseiKakuGo-W5"))

In [21]:
make_page3(meitestsu_dat.head(),cn=meitetsu_cn)

  0%|          | 0/5 [00:00<?, ?it/s]

### 画像処理

In [22]:
import os
import fitz  # PyMuPDFをインポート
from PIL import Image  # Pillowをインポート

def convert_pdf_to_image(pdf_path, output_dir, desired_width_mm, desired_height_mm, dpi=300):
    # 出力ディレクトリが存在しない場合は作成
    os.makedirs(output_dir, exist_ok=True)

    # PDFファイルを開く
    doc = fitz.open(pdf_path)

    # 物理サイズからピクセル数を計算
    width_px = int((desired_width_mm / 25.4) * dpi)
    height_px = int((desired_height_mm / 25.4) * dpi)

    for page_number in range(len(doc)):
        page = doc.load_page(page_number)

        rect = page.rect
        width_pt = rect.width
        height_pt = rect.height

        # ズーム倍率を計算
        zoom_x = width_px / width_pt
        zoom_y = height_px / height_pt

        matrix = fitz.Matrix(zoom_x, zoom_y)
        pix = page.get_pixmap(matrix=matrix, alpha=False)

        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # 画像の保存パスを作成
        image_name = f'{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{page_number + 1}.png'
        image_path = os.path.join(output_dir, image_name)
        img.save(image_path, dpi=(dpi, dpi))

    

# # 複数のPDFファイルを処理する例
# pdf_dir = 'path/to/pdf_directory'  # PDFファイルが保存されているディレクトリ
# output_dir = 'path/to/output_directory'  # 画像を保存するディレクトリ

# desired_width_mm = 420 * 0.8  # 幅をA3の0.8倍に設定
# desired_height_mm = 247 * 0.8  # 高さをA3の0.8倍に設定
# dpi = 300  # 解像度を300 DPIに設定

# # PDFディレクトリ内のすべてのPDFファイルを処理
# for filename in os.listdir(pdf_dir):
#     if filename.lower().endswith('.pdf'):
#         pdf_path = os.path.join(pdf_dir, filename)
#         convert_pdf_to_image(pdf_path, output_dir, desired_width_mm, desired_height_mm, dpi)



In [23]:


# 書き換えテスト
pdf_dir = 'workspace/page3'  # PDFファイルが保存されているディレクトリ
output_dir = 'workspace/page3_jpeg'  # 画像を保存するディレクトリ

desired_width_mm = 420 * 0.8  # 幅をA3の0.8倍に設定
desired_height_mm = 247 * 0.8  # 高さをA3の0.8倍に設定
dpi = 300  # 解像度を300 DPIに設定

# PDFディレクトリ内のすべてのPDFファイルを処理
for filename in tqdm(os.listdir(pdf_dir)):
    if filename.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, filename)
        convert_pdf_to_image(pdf_path, output_dir, desired_width_mm, desired_height_mm, dpi)



  0%|          | 0/5 [00:00<?, ?it/s]

In [24]:
# 画像パス
meitestsu_dat["画像パス"] = "workspace/page3_jpeg/" +meitestsu_dat["Evaluatee Flag ID"] +"_page3_page_1.png"


### Excel処理

In [25]:


# ループでコピー元を作る
# 出力ディレクトリが存在しない場合は作成
os.makedirs("workspace/Excel_p3", exist_ok=True)

for i,data in tqdm(meitestsu_dat.iterrows()):
    F_id = data["Evaluatee Flag ID"]
    name = data["氏名"]
    C_id = data["社員番号"]
    dept = data["所属"]
    grade = data["役職分類\n(社内)"]
    shutil.copy(f'workspace/【名鉄】カスタマイズレポート_Excel調整_最終_望月編集_p3.xlsx',f'workspace/Excel_p3/{F_id}_p3.xlsx')

0it [00:00, ?it/s]

In [26]:
# コピーしたものに記入&画像貼り付け



def excel_creation_p3(comp_df,image_path_column,image_width=None, image_height=None):
    # 出力ディレクトリが存在しない場合は作成
    os.makedirs("workspace/page3_個人Excel", exist_ok=True)

    # 該当IDのExcelファイルを開く
    for i,data in tqdm(comp_df.iterrows()):
        F_id = data["Evaluatee Flag ID"]
        name = data["氏名"]
        C_id = data["社員番号"]
        dept = data["所属"]
        grade = data["役職分類\n(社内)"]
        image_path = data[image_path_column]  
        # 長いやつ折り返しテスト
        if dept == "名鉄グループバスホールディングス":
            dept = "名鉄グループバス\nホールディングス"

        # Excel指定
        wb = openpyxl.load_workbook(f"workspace/Excel_p3/{F_id}_p3.xlsx")
        # シートを選択する
        sheet1 = wb['P3 個人結果チャート']

        # 基本情報
        sheet1["C5"] = name
        sheet1["G5"] = C_id
        sheet1["K5"] = dept
        sheet1["O5"] = grade
        
        # 画像を貼り付ける
        if image_path:  # 画像パスが存在する場合のみ
            img = open_Image(image_path)

            # サイズの調整
            if image_width:
                img.width = image_width
            if image_height:
                img.height = image_height

            # 画像の貼り付け位置を指定
            img.anchor = "C8"  # 画像を貼り付けるセル（調整可能）
            sheet1.add_image(img)

        

        # Excelファイルを保存する
        wb.save(f'workspace/page3_個人Excel/{F_id}_p3.xlsx')

In [27]:
excel_creation_p3(meitestsu_dat.head(),"画像パス",image_width=1276.5708, image_height=788
)

0it [00:00, ?it/s]

## page2

In [28]:
# ループでコピー元を作る
# 出力ディレクトリが存在しない場合は作成
os.makedirs("workspace/Excel_p2", exist_ok=True)

for i,data in tqdm(meitestsu_dat.iterrows()):
    F_id = data["Evaluatee Flag ID"]
    name = data["氏名"]
    C_id = data["社員番号"]
    dept = data["所属"]
    grade = data["役職分類\n(社内)"]
    shutil.copy(f'workspace/【名鉄】カスタマイズレポート_Excel調整_最終_望月編集_p2.xlsx',f'workspace/Excel_p2/{F_id}_p2.xlsx')

0it [00:00, ?it/s]

In [29]:
# コピーしたものに記入&画像貼り付け

import openpyxl
from openpyxl.drawing.image import Image as open_Image
from openpyxl.styles import PatternFill


def excel_creation_p2(comp_df):
    # 出力ディレクトリが存在しない場合は作成
    os.makedirs("workspace/page2_個人Excel", exist_ok=True)
    # 該当IDのExcelファイルを開く
    for i,data in tqdm(comp_df.iterrows()):
        F_id = data["Evaluatee Flag ID"]
        name = data["氏名"]
        C_id = data["社員番号"]
        dept = data["所属"]
        grade = data["役職分類\n(社内)"]
        grade_IGS = data["grade_IGS調整"]

        # 長いやつ折り返しテスト
        if dept == "名鉄グループバスホールディングス":
            dept = "名鉄グループバス\nホールディングス"


        # Excel指定
        wb = openpyxl.load_workbook(f"workspace/Excel_p2/{F_id}_p2.xlsx")
        # シートを選択する
        sheet1 = wb['P2 個人結果スコア']

        # 基本情報
        sheet1["C5"] = name
        sheet1["G5"] = C_id
        sheet1["K5"] = dept
        sheet1["O5"] = grade

        # スコア
        for comp in range(len(meitetsu_cn)):
            sheet1[f"K{comp*2+11}"] = int(round(data[meitetsu_cn[comp]],0))

        # gradeごとだしわけ
        sheet1["N10"] = grade_IGS
        # diff_list = []
        for comp in range(len(meitetsu_cn)):
            sheet1[f"N{comp*2+11}"] = int(round(grade_median_table.loc[grade_IGS,meitetsu_cn[comp]],0))
            if int(round(data[meitetsu_cn[comp]],0)) > int(round(grade_median_table.loc[grade_IGS,meitetsu_cn[comp]],0)):
                sheet1[f"O{comp*2+11}"] = "↑"
                sheet1[f"O{comp*2+11}"].fill = PatternFill(start_color="BFFFBF", end_color="BFFFBF", fill_type="solid")

            elif int(round(data[meitetsu_cn[comp]],0)) < int(round(grade_median_table.loc[grade_IGS,meitetsu_cn[comp]],0)):
                sheet1[f"O{comp*2+11}"] = "↓"
                sheet1[f"O{comp*2+11}"].fill = PatternFill(start_color="FFBFBF", end_color="FFBFBF", fill_type="solid")
                
            else:
                sheet1[f"O{comp*2+11}"] = "-"
                sheet1[f"O{comp*2+11}"].fill = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")

        # gradeごとだしわけ_bench
        grade_bench_table = compare_bench_dict[grade_IGS]
        sheet1["P10"] = grade_IGS
        # diff_list = []
        for comp in range(len(meitetsu_cn)):
            sheet1[f"P{comp*2+11}"] = int(round(grade_bench_table.loc["中央閾値",meitetsu_cn[comp]],0))
            if int(round(data[meitetsu_cn[comp]],0)) > int(round(grade_bench_table.loc["中央閾値",meitetsu_cn[comp]],0)):
                sheet1[f"Q{comp*2+11}"] = "↑"
                sheet1[f"Q{comp*2+11}"].fill = PatternFill(start_color="BFFFBF", end_color="BFFFBF", fill_type="solid")

            elif int(round(data[meitetsu_cn[comp]],0)) < int(round(grade_bench_table.loc["中央閾値",meitetsu_cn[comp]],0)):
                sheet1[f"Q{comp*2+11}"] = "↓"
                sheet1[f"Q{comp*2+11}"].fill = PatternFill(start_color="FFBFBF", end_color="FFBFBF", fill_type="solid")
                
            else:
                sheet1[f"Q{comp*2+11}"] = "-"
                sheet1[f"Q{comp*2+11}"].fill = PatternFill(start_color="C0C0C0", end_color="C0C0C0", fill_type="solid")

            

        # Excelファイルを保存する
        wb.save(f'workspace/page2_個人Excel/{F_id}_p2.xlsx')

In [30]:
excel_creation_p2(meitestsu_dat.head())

0it [00:00, ?it/s]

# Excel_to_PDF がうまくいかないので別対応

- LibreOffice使うのが良さそう
- 現状Desktopのローカル対応
- 画像貼り付けずれるので要注意

In [31]:
# Libre出力用フォルダの作成
os.makedirs("workspace/page2_PDFver", exist_ok=True)
os.makedirs("workspace/page3_PDFver", exist_ok=True)


1. スクリプトファイルを任意の場所に保存（例：`workspace/convert_excel_to_pdf.sh`）
2. 実行権限を付与  
   ```bash
   chmod +x convert_excel_to_pdf.sh
   ```
3. スクリプト内の以下3つの変数を自分の環境に合わせて修正
- INPUT_DIR：Excelファイルがあるフォルダパス
- OUTPUT_DIR：PDFを出力したいフォルダパス
- LIBREOFFICE_PATH：LibreOffice実行ファイル（Macの場合は /Applications/.../soffice）

4. 実行
```bash
./convert_excel_to_pdf.sh
```

## うまくできてないデータ抽出

In [34]:
# Page2
# Specify the folder path
folder_path = "workspace/page2_PDFver"

# Get the list of files and their sizes
file_list_p2 = pd.DataFrame()
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path)
        file_list_p2 = pd.concat([file_list_p2,pd.DataFrame([[file_name,file_size]])])

file_list_p2.columns = ["name","size"]

ID_list = []
for i in file_list_p2["name"]:
    ID_list.append(i[:9])
file_list_p2["Evaluatee Flag ID"] = ID_list

# set_check
target_list_p2 = list(set(file_list_p2["Evaluatee Flag ID"]) ^ set(meitestsu_dat["Evaluatee Flag ID"]))

folder_path = "workspace/page2_個人Excel"
excel_p2_list = os.listdir(folder_path)

excel_dat_p2 = pd.DataFrame(excel_p2_list,columns=["name"])
ID_list = []
for i in excel_dat_p2["name"]:
    ID_list.append(i[:9])
excel_dat_p2["Evaluatee Flag ID"] = ID_list

excel_dat_p2["Flag"] = excel_dat_p2["Evaluatee Flag ID"].apply(lambda x: 1 if x in target_list_p2 else 0)


# コピー元とコピー先のフォルダ
source_folder = "workspace/page2_個人Excel"
destination_folder = "workspace/re_make/page2_個人Excel"

# コピー先フォルダを作成（存在しない場合）
os.makedirs(destination_folder, exist_ok=True)

# Flagが1のファイルだけをコピー
for index, row in excel_dat_p2.iterrows():
    if row["Flag"] == 1:
        source_path = os.path.join(source_folder, row["name"])
        destination_path = os.path.join(destination_folder, row["name"])
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)

"Flag == 1 のファイルをコピーしました。"

'Flag == 1 のファイルをコピーしました。'

In [35]:
# Page3
# Specify the folder path
folder_path = "workspace/page3_PDFver"

# Get the list of files and their sizes
file_list_p3 = pd.DataFrame()
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    if os.path.isfile(file_path):
        file_size = os.path.getsize(file_path)
        file_list_p3 = pd.concat([file_list_p3,pd.DataFrame([[file_name,file_size]])])

file_list_p3.columns = ["name","size"]

ID_list = []
for i in file_list_p3["name"]:
    ID_list.append(i[:9])
file_list_p3["Evaluatee Flag ID"] = ID_list

# set_check
target_list_p3 = list(set(file_list_p3["Evaluatee Flag ID"]) ^ set(meitestsu_dat["Evaluatee Flag ID"]))

folder_path = "workspace/page3_個人Excel"
excel_p3_list = os.listdir(folder_path)

excel_dat_p3 = pd.DataFrame(excel_p3_list,columns=["name"])
ID_list = []
for i in excel_dat_p3["name"]:
    ID_list.append(i[:9])
excel_dat_p3["Evaluatee Flag ID"] = ID_list

excel_dat_p3["Flag"] = excel_dat_p3["Evaluatee Flag ID"].apply(lambda x: 1 if x in target_list_p3 else 0)


# コピー元とコピー先のフォルダ
source_folder = "workspace/page3_個人Excel"
destination_folder = "workspace/re_make/page3_個人Excel"

# コピー先フォルダを作成（存在しない場合）
os.makedirs(destination_folder, exist_ok=True)

# Flagが1のファイルだけをコピー
for index, row in excel_dat_p3.iterrows():
    if row["Flag"] == 1:
        source_path = os.path.join(source_folder, row["name"])
        destination_path = os.path.join(destination_folder, row["name"])
        if os.path.exists(source_path):
            shutil.copy(source_path, destination_path)

"Flag == 1 のファイルをコピーしました。"

'Flag == 1 のファイルをコピーしました。'

# concat

In [36]:
def pdf_merge(df):
    os.makedirs("deliverables/all_report/", exist_ok=True)

    for index, data in tqdm(df.iterrows(), total=len(df)):
        ID = data["Evaluatee Flag ID"]
        name = data["氏名"]
        C_id = data["社員番号"]
        dept = data["所属"]
        grade = data["役職分類\n(社内)"]
        grade_IGS = data["grade_IGS調整"]

        # ✅ PdfMerger を使用
        merger = PdfMerger()

        # 各ページを順番に追加
        merger.append("workspace/1217_P1【名鉄】カスタマイズレポート_Excel調整_最終.pdf")  # 共通P1
        merger.append(f"workspace/page2_PDFver/{ID}_p2.pdf")
        merger.append(f"workspace/page3_PDFver/{ID}_p3.pdf")
        merger.append("workspace/1217_P4【名鉄】カスタマイズレポート_Excel調整_最終.pdf")  # 共通P4

        # 出力ファイルパスを指定して保存
        output_path = f"deliverables/all_report/{dept}_{C_id}_{name}_GROW‐feedback-report2024.pdf"
        merger.write(output_path)
        merger.close()


In [37]:
pdf_merge(meitestsu_dat.head())

  0%|          | 0/5 [00:00<?, ?it/s]